In [1]:
import pandas as pd
import keras
import gc
import numpy as np

/home/ubuntu/workspace/tools/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from utils import *

In [3]:
mname = 'nn'

In [4]:
def load(len_train=184903890, k = 23333333):
    import pickle
    with open('../data/features.pickle', 'rb') as handle:
        df = pickle.load(handle)
    categorical_max_values = [df[c].max() for c in categorical_names]  
    df_train = df[: len_train - k]
    df_val = df[len_train - k: len_train]
    df_sub = df[len_train: ]
    print('df_train.shape %s,%s' % df_train.shape)
    print('df_val.shape %s,%s' % df_val.shape)
    print('df_sub.shape %s,%s' % df_sub.shape)
    del df
    gc.collect()
    return df_train, df_val, df_sub, categorical_max_values

df_train, df_val, df_sub, categorical_max_values = load()

df_train.shape 161570557,13
df_val.shape 23333333,13
df_sub.shape 18790469,13


In [13]:
def get_keras_data(df, y=True):
    X = {xname: np.array(df[xname]) for xname in xnames}
    if y:
        return X, df[yname]
    else:
        return X
X_train, y_train = get_keras_data(df_train.sample(frac=0.5))
X_val, y_val = get_keras_data(df_val)

In [14]:
print ('neural network....')
from keras.layers import Input, Embedding, Dense, Flatten, Dropout, concatenate
from keras.layers import BatchNormalization, SpatialDropout1D
from keras.callbacks import Callback
from keras.models import Model
from keras.optimizers import Adam

neural network....


In [15]:
emb_n = 50
dense_n = 1000
inputs = []
fes = []
for name, max_value in zip(categorical_names, categorical_max_values):
    inp = Input(shape=[1], name=name)
    inputs.append(inp)
    emb = Embedding(max_value + 1, emb_n, embeddings_initializer='uniform')(inp)
    fes.append(Flatten()(emb))
for name in xnames:
    if name not in categorical_names:
        inp = Input(shape=[1], name=name)
        inputs.append(inp)
        fes.append(inp)
fe = concatenate(fes)

In [16]:
layer0 = Dropout(0.2)(Dense(dense_n,activation='relu')(fe))
layer1 = Dropout(0.2)(Dense(dense_n,activation='relu')(layer0))
outp = Dense(1,activation='sigmoid')(layer1)
r = Model(inputs=inputs, outputs=outp)

In [17]:
batch_size = 20000
epochs = 2

exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(df_train.shape[0] / batch_size) * epochs
lr_init, lr_fin = 0.001, 0.0001
lr_decay = exp_decay(lr_init, lr_fin, steps)
optimizer_adam = Adam(lr=0.001, decay=lr_decay)
r.compile(loss='binary_crossentropy',optimizer=optimizer_adam,metrics=['accuracy'])

In [18]:
r.fit(X_train, y_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True)

Epoch 1/1
 1540000/80785278 [..............................] - ETA: 47:55 - loss: 0.1625 - acc: 0.9857

KeyboardInterrupt: 

In [27]:
r.save('../model/%s.h5' % mname)

In [28]:
y_val_pred = r.predict(X_val, batch_size=20000, verbose=1)[:, 0]

23333333/23333333 [==============================] - 30s 1us/step


In [29]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_val, y_val_pred))

0.9161105355723811

In [77]:
import pickle
print('writing val ...')
output_val = pd.DataFrame({'y_pred': y_val_pred, 'y': y_val.astype(int).values})
with open('../data/%s_val.pickle' % mname, 'wb') as handle:
    pickle.dump(output_val, handle, protocol=pickle.HIGHEST_PROTOCOL)

writing val ...


In [80]:
ids = df_sub['click_id'].astype(int).values
X_sub = get_keras_data(df_sub, y=False)
print('predicting...')
y_sub = r.predict(X_sub, batch_size=1024, verbose=1)[:, 0]
print('writing...')
output = pd.DataFrame({'click_id': ids, 'is_attributed': y_sub})
output.to_csv("../data/%s.csv.gz" % mname, compression='gzip', index=False)

predicting...
  868352/18790469 [>.............................] - ETA: 3:40

KeyboardInterrupt: 